In [1]:
import argparse
from math import log
from random import uniform

mu1 = 5
mu2 = 110.379
lmbd = 9.5
normalizing_factor = 0
accuracy = 100

P00 = 0
P10 = 0
P01 = 0
P11 = 0

class Channel:
    def __init__(self, intensity):
        self._intensity = intensity
        self._ticks_for_process = 0
        self.work_time = 0
        self.processed_claims = 0
        self.is_first_claim = True

    def add(self):
        self._ticks_for_process = exponential_number(self._intensity)
        if self.is_first_claim:
            self.is_first_claim = False
        else:
            self.processed_claims += 1

    def is_processed(self):
        return self._ticks_for_process <= 0

    def tick(self):
        if self._ticks_for_process > 0:
            self._ticks_for_process -= 1
            self.work_time += 1

class Generator:
    def __init__(self, intensity):
        self._intensity = intensity
        self._ticks_for_generate = 0

    def start_generate(self):
        self._ticks_for_generate = exponential_number(self._intensity)

    def is_generated(self):
        return self._ticks_for_generate <= 0

    def tick(self):
        if self._ticks_for_generate > 0:
            self._ticks_for_generate -= 1

def exponential_number(number):
    return (-1 / number) * log(uniform(0, 1))


def normailze_inputs():
    global mu1
    global mu2
    global lmbd
    global normalizing_factor
    normalizing_factor = max(mu1, mu2, lmbd) * accuracy
    mu1 = mu1 / normalizing_factor
    mu2 = mu2 / normalizing_factor
    lmbd = lmbd / normalizing_factor


def main():
    global P00
    global P01
    global P10
    global P11
    normailze_inputs()

    first_chanel = Channel(mu1)
    second_chanel = Channel(mu2)
    generator = Generator(lmbd)
    ticks_number = 100000 * accuracy
    declined_claims = 0
    generated_claims = 0
    processed_claims = 0
    for i in range(0, ticks_number):
        if first_chanel.is_processed() and second_chanel.is_processed():
            P00 += 1
        elif not first_chanel.is_processed() and second_chanel.is_processed():
            P10 += 1
        elif first_chanel.is_processed() and not second_chanel.is_processed():
            P01 += 1
        else:
            P11 += 1

        if generator.is_generated():
            generator.start_generate()
            generated_claims += 1
            if first_chanel.is_processed():
                first_chanel.add()
                processed_claims += 1
            elif second_chanel.is_processed():
                second_chanel.add()
                processed_claims += 1
            else:
                declined_claims += 1
        first_chanel.tick()
        second_chanel.tick()
        generator.tick()

    print('P00: ', P00 / ticks_number)
    print('P10: ', P10 / ticks_number)
    print('P01: ', P01 / ticks_number)
    print('P11: ', P11 / ticks_number)

if __name__ == "__main__":
    main()

P00:  0.3369728
P10:  0.6117781
P01:  0.0022838
P11:  0.0489653
